In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os

In [3]:
import mdtraj as md
import parmed as pmd

In [4]:
# OpenMM Imports
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as u

In [5]:
from parmed import load_file, unit as u
from parmed.openmm import StateDataReporter, NetCDFReporter

In [6]:
from paprika import align, restraints, simulate, utils

2017-11-15 02:47:16 PM OpenMM support: Yes


# Manual OpenMM run through...

## System setup (standard OpenMM code)

In [32]:
print('Creating OpenMM System')
cb6_but = load_file('../test/cb6-but/vac.topo', '../test/cb6-but/vac.crds')
system = cb6_but.createSystem(nonbondedMethod=app.NoCutoff,
                               constraints=app.HBonds, implicitSolvent=app.GBn2,
                               implicitSolventSaltConc=0.1*u.moles/u.liter,
)

# Create the integrator to do Langevin dynamics
integrator = mm.LangevinIntegrator(
                        300*u.kelvin,       # Temperature of heat bath
                        1.0/u.picoseconds,  # Friction coefficient
                        2.0*u.femtoseconds, # Time step
)

# Define the platform to use; CUDA, OpenCL, CPU, or Reference. Or do not specify
# the platform to use the default (fastest) platform
platform = mm.Platform.getPlatformByName('CUDA')
prop = dict(CudaPrecision='mixed') # Use mixed single/double precision

# Create the Simulation object
sim = app.Simulation(cb6_but.topology, system, integrator, platform, prop)

# Set the particle positions
sim.context.setPositions(cb6_but.positions)

Creating OpenMM System


## Minimize

In [15]:
# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(maxIterations=500)

Minimizing energy


In [16]:
st = sim.context.getState(getPositions=True,getEnergy=True)
print(st.getPotentialEnergy())

-3489.7370789646575 kJ/mol


In [17]:
sim.reporters.append(app.PDBReporter('minimized_structure.pdb', 1))
sim.step(1)

In [24]:
starting = md.load('vac.pdb')
final = md.load('minimized_structure.pdb')
print('RMSD = {0:1.5f} A'.format(md.rmsd(starting, final)[0]))

RMSD = 0.00492 A


# OpenMM with single atom restratints (from pAPRika)

In [203]:
rest1 = restraints.DAT_restraint()
rest1.continuous_apr = True
rest1.structure_file = 'vac.pdb'
rest1.mask1 = ':CB6@O'
rest1.mask2 = ':BUT@C3'
rest1.attach['target'] = 3.0
rest1.attach['fraction_list'] = [0.00, 0.04, 0.181, 0.496, 1.000]
rest1.attach['fc_final'] = 5.0
rest1.pull['fc'] = rest1.attach['fc_final']
rest1.pull['target_initial'] = rest1.attach['target']
rest1.pull['target_final'] = 20.0
rest1.pull['num_windows'] = 11
rest1.release['target'] = rest1.pull['target_final']
rest1.release['num_windows'] = len(rest1.attach['fraction_list'])
rest1.release['fc_initial'] = rest1.attach['fc_final']
rest1.release['fc_final'] = rest1.attach['fc_final']
rest1.initialize()

2017-11-16 10:42:52 AM Calculating attach targets and force constants...
2017-11-16 10:42:52 AM Method #3
2017-11-16 10:42:52 AM Calculating pull targets and force constants...
2017-11-16 10:42:52 AM Method #1
2017-11-16 10:42:52 AM Calculating release targets and force constants...
2017-11-16 10:42:52 AM Method #1
2017-11-16 10:42:52 AM Number of attach windows = 5
2017-11-16 10:42:52 AM Number of pull windows = 11
2017-11-16 10:42:52 AM Number of release windows = 5
2017-11-16 10:42:52 AM Assigning atom indices ...
2017-11-16 10:42:52 AM Loaded vac.pdb...
2017-11-16 10:42:52 AM There are 1 atoms in the mask :CB6@O  ...
2017-11-16 10:42:52 AM Loaded vac.pdb...
2017-11-16 10:42:52 AM There are 1 atoms in the mask :BUT@C3  ...


In [204]:
print('Creating OpenMM System')
cb6_but = load_file('../test/cb6-but/vac.topo', '../test/cb6-but/vac.crds')
system = cb6_but.createSystem(nonbondedMethod=app.NoCutoff,
                               constraints=app.HBonds, implicitSolvent=app.GBn2,
                               implicitSolventSaltConc=0.1*u.moles/u.liter,
)

# Create the integrator to do Langevin dynamics
integrator = mm.LangevinIntegrator(
                        300*u.kelvin,       # Temperature of heat bath
                        1.0/u.picoseconds,  # Friction coefficient
                        2.0*u.femtoseconds, # Time step
)

# Define the platform to use; CUDA, OpenCL, CPU, or Reference. Or do not specify
# the platform to use the default (fastest) platform
platform = mm.Platform.getPlatformByName('CUDA')
prop = dict(CudaPrecision='mixed') 

Creating OpenMM System


In [205]:
phase = 'pull'
window = 10

print('Trying to move {} and {} to {} A with force constant {} kcal/mol/A**2'.format(
    rest1.index1[0],
    rest1.index2[0],
    rest1.phase[phase]['targets'][window],
    rest1.phase[phase]['force_constants'][window]))

bond_restraint = mm.CustomBondForce('k * (r - r_0)^2')
bond_restraint.addPerBondParameter('k')
bond_restraint.addPerBondParameter('r_0')

r_0 = rest1.phase[phase]['targets'][window] * 0.1 * u.nanometers
k  = rest1.phase[phase]['force_constants'][window] / 0.239 / 0.01 * u.kilojoules_per_mole/u.nanometers**2

print(r_0, k)

bond_restraint.addBond(rest1.index1[0], rest1.index2[0],
                       [k, r_0])

system.addForce(bond_restraint)

# Create the Simulation object
sim = app.Simulation(cb6_but.topology, system, integrator, platform, prop)

# Set the particle positions
sim.context.setPositions(cb6_but.positions)

Trying to move 13 and 119 to 20.0 A with force constant 5.0 kcal/mol/A**2
2.0 nm 2092.050209205021 kJ/(nm**2 mol)


In [206]:
# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(maxIterations=500)

Minimizing energy


In [207]:
positions = sim.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(sim.topology, positions, open('minimized_with_restraint.pdb', 'w'))

# OpenMM with a group restratint (from pAPRika)

In [217]:
rest1 = restraints.DAT_restraint()
rest1.continuous_apr = True
rest1.structure_file = 'vac.pdb'
rest1.mask1 = ':CB6@O'
rest1.mask2 = ':BUT@C*'
rest1.attach['target'] = 3.0
rest1.attach['fraction_list'] = [0.00, 0.04, 0.181, 0.496, 1.000]
rest1.attach['fc_final'] = 5.0
rest1.pull['fc'] = rest1.attach['fc_final']
rest1.pull['target_initial'] = rest1.attach['target']
rest1.pull['target_final'] = 10.0
rest1.pull['num_windows'] = 11
rest1.release['target'] = rest1.pull['target_final']
rest1.release['num_windows'] = len(rest1.attach['fraction_list'])
rest1.release['fc_initial'] = rest1.attach['fc_final']
rest1.release['fc_final'] = rest1.attach['fc_final']
rest1.initialize()

2017-11-16 10:45:29 AM Calculating attach targets and force constants...
2017-11-16 10:45:29 AM Method #3
2017-11-16 10:45:29 AM Calculating pull targets and force constants...
2017-11-16 10:45:29 AM Method #1
2017-11-16 10:45:29 AM Calculating release targets and force constants...
2017-11-16 10:45:29 AM Method #1
2017-11-16 10:45:29 AM Number of attach windows = 5
2017-11-16 10:45:29 AM Number of pull windows = 11
2017-11-16 10:45:29 AM Number of release windows = 5
2017-11-16 10:45:29 AM Assigning atom indices ...
2017-11-16 10:45:29 AM Loaded vac.pdb...
2017-11-16 10:45:29 AM There are 1 atoms in the mask :CB6@O  ...
2017-11-16 10:45:29 AM Loaded vac.pdb...
2017-11-16 10:45:29 AM There are 4 atoms in the mask :BUT@C*  ...


In [223]:
print('Creating OpenMM System')
cb6_but = load_file('../test/cb6-but/vac.topo', '../test/cb6-but/vac.crds')
system = cb6_but.createSystem(nonbondedMethod=app.NoCutoff,
                               constraints=app.HBonds, implicitSolvent=app.GBn2,
                               implicitSolventSaltConc=0.1*u.moles/u.liter,
)

integrator = mm.LangevinIntegrator(
                        300*u.kelvin,       # Temperature of heat bath
                        1.0/u.picoseconds,  # Friction coefficient
                        2.0*u.femtoseconds, # Time step
)

platform = mm.Platform.getPlatformByName('CUDA')
prop = dict(CudaPrecision='mixed') 

Creating OpenMM System


In [224]:
phase = 'pull'
window = 10

print('Trying to move {} and {} to {} A with force constant {} kcal/mol/A**2'.format(
    rest1.index1[0],
    rest1.index2,
    rest1.phase[phase]['targets'][window],
    rest1.phase[phase]['force_constants'][window]))

Trying to move 13 and [109, 113, 115, 119] to 10.0 A with force constant 5.0 kcal/mol/A**2


In [225]:
bond_restraint = mm.CustomCentroidBondForce(2, 'k * (distance(g1, g2) - r_0)^2')
bond_restraint.addPerBondParameter('k')
bond_restraint.addPerBondParameter('r_0')

r_0 = rest1.phase[phase]['targets'][window] * u.angstroms
k  = rest1.phase[phase]['force_constants'][window] * u.kilocalories_per_mole/u.angstroms**2

print(r_0, k)

g1 = bond_restraint.addGroup(rest1.index1)
g2 = bond_restraint.addGroup(rest1.index2)

print(rest1.index1, rest1.index2)

print(g1, g2)

bond_restraint.addBond([g1, g2],
                       [k, r_0])

system.addForce(bond_restraint)

# Create the Simulation object
sim = app.Simulation(cb6_but.topology, system, integrator, platform, prop)

# Set the particle positions
sim.context.setPositions(cb6_but.positions)

10.0 A 5.0 kcal/(A**2 mol)
[13] [109, 113, 115, 119]
0 1


In [226]:
# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(maxIterations=500)

Minimizing energy


In [227]:
positions = sim.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(sim.topology, positions, open('minimized_with_com_restraint.pdb', 'w'))

In [214]:
# sim.reporters.append(app.PDBReporter('minimized_with_com_restraint.pdb', 1))
# sim.reporters.append(app.DCDReporter('tmp.dcd', 1))
# sim.step(10000)

# OpenMM with two group restraints (from pAPRika)

In [237]:
rest1 = restraints.DAT_restraint()
rest1.continuous_apr = True
rest1.structure_file = 'vac.pdb'
rest1.mask1 = ':CB6@*'
rest1.mask2 = ':BUT@*'
rest1.attach['target'] = 3.0
rest1.attach['fraction_list'] = [0.00, 0.04, 0.181, 0.496, 1.000]
rest1.attach['fc_final'] = 5.0
rest1.pull['fc'] = rest1.attach['fc_final']
rest1.pull['target_initial'] = rest1.attach['target']
rest1.pull['target_final'] = 10.0
rest1.pull['num_windows'] = 11
rest1.release['target'] = rest1.pull['target_final']
rest1.release['num_windows'] = len(rest1.attach['fraction_list'])
rest1.release['fc_initial'] = rest1.attach['fc_final']
rest1.release['fc_final'] = rest1.attach['fc_final']
rest1.initialize()

2017-11-16 11:25:46 AM Calculating attach targets and force constants...
2017-11-16 11:25:46 AM Method #3
2017-11-16 11:25:46 AM Calculating pull targets and force constants...
2017-11-16 11:25:46 AM Method #1
2017-11-16 11:25:46 AM Calculating release targets and force constants...
2017-11-16 11:25:46 AM Method #1
2017-11-16 11:25:46 AM Number of attach windows = 5
2017-11-16 11:25:46 AM Number of pull windows = 11
2017-11-16 11:25:46 AM Number of release windows = 5
2017-11-16 11:25:46 AM Assigning atom indices ...
2017-11-16 11:25:46 AM Loaded vac.pdb...
2017-11-16 11:25:46 AM There are 108 atoms in the mask :CB6@*  ...
2017-11-16 11:25:46 AM Loaded vac.pdb...
2017-11-16 11:25:46 AM There are 14 atoms in the mask :BUT@*  ...


In [238]:
print('Creating OpenMM System')
cb6_but = load_file('../test/cb6-but/vac.topo', '../test/cb6-but/vac.crds')
system = cb6_but.createSystem(nonbondedMethod=app.NoCutoff,
                               constraints=app.HBonds, implicitSolvent=app.GBn2,
                               implicitSolventSaltConc=0.1*u.moles/u.liter,
)

integrator = mm.LangevinIntegrator(
                        300*u.kelvin,       # Temperature of heat bath
                        1.0/u.picoseconds,  # Friction coefficient
                        2.0*u.femtoseconds, # Time step
)

platform = mm.Platform.getPlatformByName('CUDA')
prop = dict(CudaPrecision='mixed') 

Creating OpenMM System


In [239]:
phase = 'pull'
window = 10

print('Trying to move {} and {} to {} A with force constant {} kcal/mol/A**2'.format(
    rest1.index1,
    rest1.index2,
    rest1.phase[phase]['targets'][window],
    rest1.phase[phase]['force_constants'][window]))

Trying to move [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108] and [109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122] to 10.0 A with force constant 5.0 kcal/mol/A**2


In [240]:
bond_restraint = mm.CustomCentroidBondForce(2, 'k * (distance(g1, g2) - r_0)^2')
bond_restraint.addPerBondParameter('k')
bond_restraint.addPerBondParameter('r_0')

r_0 = rest1.phase[phase]['targets'][window] * u.angstroms
k  = rest1.phase[phase]['force_constants'][window] * u.kilocalories_per_mole/u.angstroms**2

g1 = bond_restraint.addGroup([i-1 for i in rest1.index1])
g2 = bond_restraint.addGroup([i-1 for i in rest1.index2])


bond_restraint.addBond([g1, g2],
                       [k, r_0])

system.addForce(bond_restraint)

# Create the Simulation object
sim = app.Simulation(cb6_but.topology, system, integrator, platform, prop)

# Set the particle positions
sim.context.setPositions(cb6_but.positions)

In [241]:
# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(maxIterations=500)

Minimizing energy


In [242]:
positions = sim.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(sim.topology, positions, open('minimized_with_two_com_restraint.pdb', 'w'))